In [1]:
import numpy as np
from genetic_selection import GeneticSelectionCV
from model_training import get_X_Y
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import global_config as cfg

csv_path = cfg.padel_csvfilepath
X, blood_y, brain_y, ratio_y, SMILES = get_X_Y(csv_path)

y = ratio_y
model_type = cfg.model_type
param_name = 'blood_params'
params = cfg.model_params.get(model_type)
model = XGBRegressor()

selector = GeneticSelectionCV(
    model,
    cv=5,
    verbose=1,
    scoring="r2",
    max_features=50,
    n_population=400,
    crossover_proba=0.5,
    mutation_proba=0.2,
    n_generations=30,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.05,
    tournament_size=3,
    n_gen_no_change=10,
    caching=True,
    n_jobs=-1,
)
selector = selector.fit(X, y)



c:\Users\80473\anaconda3\envs\BBB\lib\site-packages\ipykernel_launcher.py:10: DtypeWarning: Columns (788,789,831,835,837,848,854,858,907,908,950,954,956,967,973,977,1148,1149) have mixed types.Specify dtype option on import or set low_memory=False.
  # Remove the CWD from sys.path while we load stuff.


Selecting features with genetic algorithm.
gen	nevals	avg                            	std                               	min                               	max                               
0  	400   	[-7.019355 25.8      11.761737]	[ 10.22043   14.468241  17.291038]	[-76.693551   1.         0.106093]	[  0.237724  50.        94.49206 ]
1  	251   	[-2077.913377   105.285     2079.954835]	[ 4053.680764   164.222665  4052.648446]	[-10000.            1.            0.061653]	[     0.237724    473.        10000.      ]
2  	226   	[-2077.277525   102.6825    2078.902073]	[ 4054.006044   161.782498  4053.185409]	[-10000.            1.            0.061653]	[     0.237724    464.        10000.      ]
3  	238   	[-2051.663921    99.075     2052.872964]	[ 4036.17443    157.475266  4035.567895]	[-10000.            1.            0.037591]	[     0.316198    476.        10000.      ]
4  	230   	[-1901.635264    95.5075    1902.789493]	[ 3922.220812   154.983176  3921.66867 ]	[-10000.            1.   

In [2]:
print(selector.support_)
print(selector.n_features_)
fea = []
idx = 0
for i in selector.support_:
    if i:
        fea.append(idx)
    idx = idx + 1
print(fea)

f = open('./ga_output.txt', 'w')
f.write(str(selector.support_))
f.write("\n")
f.write(str(fea))
f.close()

[False False False ... False False False]
24
[394, 786, 877, 1169, 1273, 1514, 2155, 2383, 2952, 3064, 3617, 3647, 3701, 3780, 3882, 3911, 4959, 4960, 5624, 6037, 7751, 7799, 7830, 8038]


In [3]:
X_s = X.iloc[:, selector.support_]
X_train, X_val, y_train, y_val = train_test_split(X_s, y, test_size=0.2)
model.fit(X_train, y_train)
pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, pred))
r2 = r2_score(y_val, pred)
print("RMSE: ", rmse)
print("R2: ", r2)


RMSE:  0.775406453541528
R2:  0.6882448253433532
